In [1]:
!python3.6 -m pip install tensorboardX
!python3.6 -m pip install torch
!python3.6 -m pip install librosa
!apt-get install libsndfile1 --yes

     |████████████████████████████████| 204kB 1.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 734.6MB 125kB/s  eta 0:00:01     |███████████▋                    | 266.5MB 12.2MB/s eta 0:00:39     |███████████████▉                | 364.0MB 10.5MB/s eta 0:00:36     |████████████████████████████▍   | 652.5MB 12.1MB/s eta 0:00:07
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 1.6MB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 12.2MB/s eta 0:00:01
     |████████████████████████████████| 7.1MB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 296kB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 327kB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 20.2MB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 399

Selecting previously unselected package libogg0:amd64.
(Reading database ... 15402 files and directories currently installed.)
Preparing to unpack .../libogg0_1.3.2-1_amd64.deb ...
Unpacking libogg0:amd64 (1.3.2-1) ...
Selecting previously unselected package libflac8:amd64.
Preparing to unpack .../libflac8_1.3.2-1_amd64.deb ...
Unpacking libflac8:amd64 (1.3.2-1) ...
Selecting previously unselected package libvorbis0a:amd64.
Preparing to unpack .../libvorbis0a_1.3.5-4.2_amd64.deb ...
Unpacking libvorbis0a:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libvorbisenc2:amd64.
Preparing to unpack .../libvorbisenc2_1.3.5-4.2_amd64.deb ...
Unpacking libvorbisenc2:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libsndfile1:amd64.
Preparing to unpack .../libsndfile1_1.0.28-4ubuntu0.18.04.1_amd64.deb ...
Unpacking libsndfile1:amd64 (1.0.28-4ubuntu0.18.04.1) ...
Setting up libogg0:amd64 (1.3.2-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libvo

# Test code copy

In [1]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from preprocess.tacotron.norm_utils import spectrogram2wav, get_spectrogram
from scipy.io.wavfile import write
import glob
import os
from solver import Solver
import librosa
import librosa.display
from plots import plot
%matplotlib inline

args_hps='vctk.json'
args_model="model_vctk/fixed_8bit_16k_paused_reusmed_latschediters_50k/model_vctk-80000"
args_source="test/p225_001.wav"
args_source_8="test/p225_001-8.wav"
args_target=0
args_output="test/output.wav"
args_sample_rate=16000
args_use_gen=False
wavenet_mel = True

hps = Hps()
hps.load(args_hps)
hps_tuple = hps.get_tuple()
solver = Solver(hps_tuple, None, wavenet_mel=wavenet_mel)
solver.load_model(args_model)
spec = get_spectrogram(args_source, wavenet_mel)
spec_8 = get_spectrogram(args_source_8, wavenet_mel)

spec_expand = np.expand_dims(spec_8, axis=0)
spec_tensor = torch.from_numpy(spec_expand).type(torch.FloatTensor)
c = Variable(torch.from_numpy(np.array([int(args_target)]))).cuda()
result = solver.test_step(spec_tensor, c, gen=args_use_gen)
result = result.squeeze(axis=0).transpose((1, 0))
wav_data = spectrogram2wav(result)
write(args_output, rate=args_sample_rate, data=wav_data)

plot(spec.T, spec_8.T, result.T)

load model from model_vctk/fixed_8bit_16k_paused_reusmed_latschediters_50k/model_vctk-80000


RuntimeError: Error(s) in loading state_dict for Encoder:
	size mismatch for conv1s.0.weight: copying a param with shape torch.Size([128, 513, 1]) from checkpoint, the shape in current model is torch.Size([128, 80, 1]).
	size mismatch for conv1s.1.weight: copying a param with shape torch.Size([128, 513, 2]) from checkpoint, the shape in current model is torch.Size([128, 80, 2]).
	size mismatch for conv1s.2.weight: copying a param with shape torch.Size([128, 513, 3]) from checkpoint, the shape in current model is torch.Size([128, 80, 3]).
	size mismatch for conv1s.3.weight: copying a param with shape torch.Size([128, 513, 4]) from checkpoint, the shape in current model is torch.Size([128, 80, 4]).
	size mismatch for conv1s.4.weight: copying a param with shape torch.Size([128, 513, 5]) from checkpoint, the shape in current model is torch.Size([128, 80, 5]).
	size mismatch for conv1s.5.weight: copying a param with shape torch.Size([128, 513, 6]) from checkpoint, the shape in current model

# Train code copy

In [3]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from utils import DataLoader
from utils import Logger
from utils import SingleDataset
from solver import Solver
import argparse
%matplotlib inline

args_load_model=False
args_flag='train'
args_hps_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk.json"
args_dataset_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data.h5"
args_dataset_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data-8-wavenet.h5"
args_index_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index.json"
args_index_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index.json"
args_output_model_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/model_vctk/model_vctk"
args_load_model_path="model_vctk/model_vctk-32000"
wavenet_mel=True

hps = Hps()
hps.load(args_hps_path)
hps_tuple = hps.get_tuple()
dataset = SingleDataset(args_dataset_path, args_index_path, seg_len=hps_tuple.seg_len)
dataset_trg = SingleDataset(args_dataset_path_trg, args_index_path_trg, seg_len=hps_tuple.seg_len)
data_loader = DataLoader(dataset, dataset_trg)

solver = Solver(hps_tuple, data_loader, wavenet_mel)
if args_load_model:
    solver.load_model(args_load_model_path)

solver.train(args_output_model_path, args_flag, mode='pretrain_G')
solver.train(args_output_model_path, args_flag, mode='pretrain_D')
solver.train(args_output_model_path, args_flag, mode='train')
solver.train(args_output_model_path, args_flag, mode='patchGAN')

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.